In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Dune Dashboard Base Class

> Base class for pulling data from dune

In [ ]:
#| default_exp dune

In [ ]:
#| export
import os
import dotenv
import pandas as pd
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import Query
import diskcache

In [ ]:
# | export
class DuneWrapper:
    
    def __init__(self):
        dotenv.load_dotenv()
        self.dune = DuneClient(os.environ["DUNE_API_KEY"])
        self.cache = diskcache.Cache('cache')
        
    def refresh_into_dataframe(self, query_id, params=[], name="Sample Query", update_cache=False):
        params_formatted = []
        for parameter in params:          
            params_formatted.append(getattr(QueryParameter, parameter["type"] + "_type")(name=parameter["name"], value=parameter["value"]))
            
        query = Query(
            name=name,
            query_id=query_id,
            params=params_formatted
        )

        cache_key = f'refresh_into_dataframe-{query_id}'
        if (cache_key in self.cache) and not update_cache:
            df = self.cache[cache_key]
        else:
            df = self.dune.refresh_into_dataframe(query=query)
            self.cache[cache_key] = df
        return df


In [ ]:
dune = DuneWrapper()

In [ ]:
dune.refresh_into_dataframe(1898885)

,block_date,avg_mint_price,avg_burn_price,avg_mkt_price
0,2022-01-25T00:00:00Z,1.956345888613234,<nil>,2.3050899888800456
1,2022-01-26T00:00:00Z,2.893025644547676,<nil>,2.5920101807901506
2,2022-01-27T00:00:00Z,2.864333966225624,2.2216159017515933,2.5403510974065853
3,2022-01-28T00:00:00Z,2.8263603268647635,2.24996276540192,2.4776305971040147
4,2022-01-29T00:00:00Z,2.9437308337890533,<nil>,2.683241567655875
...,...,...,...,...
443,2023-04-14T00:00:00Z,0.6202724212949381,0.5619478206375653,0.5711961515326186
444,2023-04-15T00:00:00Z,0.6112574230310933,0.5790590405319433,0.57566023154377
445,2023-04-16T00:00:00Z,0.611212703214154,0.5864501254515719,0.5686697460379618
446,2023-04-17T00:00:00Z,0.6126107071626389,0.5963675087675443,<nil>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()